# **Pre-processing technique: Reweighing for Statistical Parity**
We will illustrate how to mitigate bias in a binary classification context. Let's imagine that a company is looking to hire a new employee. They use a machine learning algorithm to select the top candidates. The candidates are assigned either 0 if they are not selected or 1 if they are. 

In this notebook, we will address unwanted bias by reweighing the dataset. Reweighing (Kamiran and Calders, 2012) is a pre-processing technique that amends the dataset to achieve statistical parity. The steps we will take are outlined below.

1. First, we will calculate Disparate Impact and Statistical Parity Difference metrics for our training dataset
2. Before we start using a predictive model, we will use the Reweighing technique on the full dataset. This step should show that with the calculated weights assigned to the dataset, disparate impact and statistical parity difference are both removed. 
3. We will then apply the Reweighing method to data used to train a predictive model and observe the results.

## **Install Libraries and load data**

In [1]:
# install AIF360
!pip install aif360
!pip install fairlearn

import pickle
import pandas as pd
import numpy as np
import seaborn as sns

from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing

from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.linear_model import RidgeClassifier


Please download the data from the following link: https://hai-data.s3.eu-west-2.amazonaws.com/roadmaps/hiring.pkl. If running in Colab, please upload the data to the local folder. Otherwise, place the data in the same folder as the notebook. Load the data into a dataframe using pickle.load and print the dataframe to inspect it.

In [2]:
with open('hiring.pkl', 'rb') as handle:
    df = pickle.load(handle)
    
df[:5] #display the first 5 candidates data

,0,1,2,3,4,5,6,7,8,9,...,497,498,499,Gender_Female,Gender_Male,Ethnicity_Asian,Ethnicity_Black,Ethnicity_Hispanic,Ethnicity_White,Action
0,28.021737,4.351153,2.453895,1.637143,-1.746628,-0.483463,0.034170,1.399225,-0.795440,0.417474,...,0.258914,-0.050558,0.014513,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,29.603342,-3.407193,0.771800,-2.957411,0.599226,-2.805277,0.329414,-2.055339,-1.194446,-0.633159,...,0.442939,-0.054423,0.026959,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,26.504283,0.642464,2.522944,-2.197094,2.270646,-0.472510,0.532815,-0.266449,-0.131638,1.038315,...,0.017111,-0.012309,0.264572,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,25.012088,0.895121,-2.092517,3.687830,0.539642,1.988930,1.121646,2.255337,-0.128801,1.148379,...,-0.149901,-0.217130,0.004403,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,27.358934,-2.332423,0.154999,-2.623793,1.682456,1.262280,-1.685565,0.489319,-0.043471,-0.372265,...,0.033429,-0.199198,0.229629,0.0,1.0,0.0,0.0,1.0,0.0,1.0


## **Calculate unwanted historic bias in our training data**




Using AIF360, we first create a [Binary Label Dataset](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.BinaryLabelDataset.html) from the training data. In order to create a Binary Label Dataset, we need the following inputs: 

*   the dataset 
*   the name of the label
*   the name of the protected attribute 

Set up variables for the privileged and unprivileged groups. In this example we will assign 'Ethnicity_White' as our privileged group. All other ethnicities are assigned as unprivileged.

In [3]:
# Compute Disparate Impact and Statistical Parity Difference metrics for the original dataset, without Reweighing

# Set up variables for the privileged and unprivileged groups

label_name = 'Action'
prot = 'Ethnicity_White'
privileged_group = [{prot: 1}] 
unprivileged_group = [{prot: 0}]

# Create a Binary Label Dataset from the training data
df_aif = BinaryLabelDataset(df=df, label_names=[label_name], protected_attribute_names=[prot])


Then, we compute Disparate Impact and Statistical Parity Difference by 
creating a  [Binary Label Dataset Metric](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.metrics.BinaryLabelDatasetMetric.html). We print the results alongside their target values.



In [4]:
# Compute metrics to estimate bias on the original dataset without Reweighing
bias_metrics = BinaryLabelDatasetMetric(df_aif, unprivileged_groups=unprivileged_group, privileged_groups=privileged_group)

print("Statistical Parity Difference : %.2f; Target value: %.2f" %(bias_metrics.statistical_parity_difference(), 0.))
print("Disparate Impact : %.2f; Target value: %.2f" %(bias_metrics.disparate_impact(), 1.))

Statistical Parity Difference : -0.04; Target value: 0.00
Disparate Impact : 0.90; Target value: 1.00


## **Apply Reweighing to the full dataset**


We now use the [Reweighing](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.algorithms.preprocessing.Reweighing.html) technique to assign weights to each training data tuple. We then calculate the Disparate Impact and Statistical Parity metrics again for the pre-processed dataset.

In [5]:
# Use Reweighing to assign weights to each training data tuple
reweighing = Reweighing(unprivileged_group, privileged_group)
w = reweighing.fit(df_aif)
df_reweighed = reweighing.transform(df_aif)

# Compute metrics on the original dataset with Reweighing
bias_metrics = BinaryLabelDatasetMetric(df_reweighed, unprivileged_groups=unprivileged_group, privileged_groups=privileged_group)

print("Statistical Parity Difference : %10.3E; Target value: %.2f" %(bias_metrics.statistical_parity_difference(), 0.))
print("Disparate Impact : %10.3E; Target value: %.2f" %(bias_metrics.disparate_impact(), 1.))

Statistical Parity Difference : -1.110E-16; Target value: 0.00
Disparate Impact :  1.000E+00; Target value: 1.00


We can see that each metric is very close to the target value, indicating that the method has been successful.

## **Run a baseline predictive model without applying reweighing**


Now, we will build a standard Ridge Classifier and observe some baseline results, using the original data without Reweighing. 

We train a Ridge Classifier with 10 fold stratified cross validation.

In [6]:
# Instantiate the classifier (this code is ready to run, there are no gaps to fill)
from sklearn import metrics

model = RidgeClassifier()

# instantiate the cross-validation scheme
mv = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)

# setup the performance metrics to be computed
perf_metrics = {"Accuracy": metrics.accuracy_score, 
                "Precision": metrics.precision_score, 
                "Recall": metrics.recall_score, 
                "F1-Score": metrics.f1_score, 
                }

In [7]:
# Train a ridge regression classifier on the dataset before reweighing 
k, i = True, 1

# instantiating X
df.columns = df.columns.astype(str)
X = df.drop(labels=df.columns[-1], axis=1)

# instantiating the target variable
y = df[df.columns[-1]]

for (train, test) in mv.split(X, y):

    # Create a Binary Label Dataset
    dataset = X.iloc[train].copy()
    dataset[df.columns[-1]] = y.iloc[train]
    dataset_BLD = BinaryLabelDataset(df=dataset, label_names=['Action'], protected_attribute_names=[prot])

    # instantiating X
    X_train = pd.DataFrame(data = dataset_BLD.features, columns=df.columns[:-1]) 

    # instantiating y
    y_train = pd.DataFrame(data = dataset_BLD.labels.ravel())

    # fit model 
    model = model.fit(X_train, y_train[0].ravel())
    
    # get predictions in the test set
    ypred_class = model.predict(X.iloc[test])

    # compute performance metrics
    metrics = []
    dataset = X.iloc[test].copy()
    dataset[df.columns[-1]] = np.expand_dims(y.iloc[test], axis=1)
    dataset = BinaryLabelDataset(df=dataset, label_names=['Action'], protected_attribute_names=[prot])
    dataset_pred = X.iloc[test].copy()
    dataset_pred[df.columns[-1]] = np.expand_dims(ypred_class, axis=1)
    dataset_pred = BinaryLabelDataset(df=dataset_pred, label_names=['Action'], protected_attribute_names=[prot])
    metric_CM = ClassificationMetric(dataset, dataset_pred, privileged_groups=privileged_group, unprivileged_groups=unprivileged_group)
    for pf in perf_metrics.keys():
            metrics += [[pf, perf_metrics[pf](y.iloc[test].values.ravel(), ypred_class)]]
    
    # Compute fairness metrics
    metrics += [['Statistical Parity Difference', metric_CM.statistical_parity_difference()]]
    metrics += [['Average Odds Difference', metric_CM.average_odds_difference()]]
    metrics += [['False Negative Rate Difference', metric_CM.false_negative_rate_difference()]]

    # concatenate results
    df_m = pd.DataFrame(metrics, columns=["Metric", "Value"])
    df_m["Fold"] = i
    i += 1
    if k:
        df_metrics_orig = df_m.copy()
        k=0
    else:
        df_metrics_orig = pd.concat([df_metrics_orig, df_m.copy()], axis=0, ignore_index=True)

Now we compute and display performance metrics (Accuracy, Precision, Recall and F1 Score) and bias metrics (Average Odds Difference, Equal Opportunity Difference, Disparate Impact, Statistical Parity Difference)

In [8]:
df_metrics_orig_tot = df_metrics_orig.groupby(by='Metric').agg(
                      {'Value':['mean','std']})

df_metrics_orig_tot

Value          
                                    mean       std
Metric                                            
Accuracy                        0.712501  0.007711
Average Odds Difference        -0.051434  0.015280
F1-Score                        0.589124  0.012598
False Negative Rate Difference  0.065355  0.031358
Precision                       0.653016  0.012121
Recall                          0.536745  0.015263
Statistical Parity Difference  -0.063249  0.012323

## **Use the Reweighed data to train a predictive model**



We now amend our Ridge Classifier training to apply Reweighing to each fold of training data. We then compare performance metrics (Accuracy, Precision, Recall and F1 Score) and fairness metrics (Average Odds Difference, Equal Opportunity Difference, Disparate Impact, Statistical Parity Difference)

In [9]:
# Train a ridge regression classifier on the dataset after reweighing
k, i = True, 1

# instantiating X
X = df.drop(labels=df.columns[-1], axis=1)

# instantiating the target variable
y = df[df.columns[-1]]

for (train, test) in mv.split(X, y):

    # TODO: Reweigh the training data
    dataset = X.iloc[train].copy()
    dataset[df.columns[-1]] = y.iloc[train]
    dataset_BLD = BinaryLabelDataset(df=dataset, label_names=['Action'], protected_attribute_names=[prot])
    dataset_BLD_reweighed = reweighing.fit_transform(dataset_BLD)

    #/ TODO
    X_train = pd.DataFrame(data = dataset_BLD_reweighed.features, columns=df.columns[:-1]) 

    # instantiating y
    y_train = pd.DataFrame(data = dataset_BLD_reweighed.labels.ravel())

    # instantiating the reweighing weights
    w_train = pd.DataFrame(data = dataset_BLD_reweighed.instance_weights.ravel())

    # fit model including sample weights calculated in reweighing
    model = model.fit(X_train, y_train[0].ravel(), sample_weight = w_train.to_numpy().ravel())
    
    # get predictions in the test set
    ypred_class = model.predict(X.iloc[test])

    # compute performance metrics
    metrics = []
    dataset = X.iloc[test].copy()
    dataset[df.columns[-1]] = np.expand_dims(y.iloc[test], axis=1)
    dataset = BinaryLabelDataset(df=dataset, label_names=['Action'], protected_attribute_names=[prot])
    dataset_pred = X.iloc[test].copy()
    dataset_pred[df.columns[-1]] = np.expand_dims(ypred_class, axis=1)
    dataset_pred = BinaryLabelDataset(df=dataset_pred, label_names=['Action'], protected_attribute_names=[prot])
    metric_CM = ClassificationMetric(dataset, dataset_pred, privileged_groups=privileged_group, unprivileged_groups=unprivileged_group)
    for pf in perf_metrics.keys():
            metrics += [[pf, perf_metrics[pf](y.iloc[test].values.ravel(), ypred_class.ravel())]]
    
    # Compute fairness metrics
    metrics += [['Statistical Parity Difference', metric_CM.statistical_parity_difference()]]
    metrics += [['Average Odds Difference', metric_CM.average_odds_difference()]]
    metrics += [['False Negative Rate Difference', metric_CM.false_negative_rate_difference()]]

    # concatenate results
    df_m = pd.DataFrame(metrics, columns=["Metric", "Value"])
    df_m["Fold"] = i
    i += 1
    if k:
        df_metrics = df_m.copy()
        k=0
    else:
        df_metrics = pd.concat([df_metrics, df_m.copy()], axis=0, ignore_index=True)

In [10]:
df_metrics_tot = df_metrics.groupby(by='Metric').agg(
                      {'Value':['mean','std']})

df_metrics_tot

Value          
                                    mean       std
Metric                                            
Accuracy                        0.713978  0.007816
Average Odds Difference        -0.007049  0.014918
F1-Score                        0.591378  0.012854
False Negative Rate Difference  0.010944  0.034097
Precision                       0.655206  0.011927
Recall                          0.538995  0.015436
Statistical Parity Difference  -0.021184  0.011293

We can see that the algorithm presents less bias towards the unprivileged group, while performance seems to remain mainly unchanged. 